In [25]:
!pip install langchain-ollama

import os, getpass

from langchain_core.callbacks import CallbackManagerForLLMRun, AsyncCallbackManagerForLLMRun
from langchain_core.outputs import ChatResult


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

os.environ['OPENAI_API_KEY'] = os.environ.get('OPEN_AI_KEY')

In [32]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
ollama_chat = ChatOllama(model="deepseek-r1:7b", temperature=0)

In [45]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="What's a sigmoid function?", name="Adam")

# Create a message list
messages = [msg]

#Invoke a model with a list of messages
gpt4o_chat.invoke(messages)
response = ollama_chat.invoke(messages)
print(f"{response.model_dump_json()}")

{"content":"<think>\n\n</think>\n\nA sigmoid function is a mathematical function that has a characteristic \"S\" shape. It is often used in various fields, including machine learning, artificial intelligence, and biology, to model processes that have an initial slow growth, followed by rapid growth, and then leveling off.\n\nOne of the most commonly used sigmoid functions is the logistic function, defined as:\n\n\\[\nf(x) = \\frac{1}{1 + e^{-x}}\n\\]\n\nThis function maps any real-valued input \\( x \\) to a value between 0 and 1, making it useful for modeling probabilities or binary outcomes.\n\nAnother common example is the hyperbolic tangent (tanh) function:\n\n\\[\nf(x) = \\frac{e^x - e^{-x}}{e^x + e^{-x}}\n\\]\n\nThis function maps inputs to values between -1 and 1, which can be useful in certain types of neural networks.\n\nThe sigmoid function is differentiable, with the derivative given by:\n\n\\[\nf'(x) = f(x)(1 - f(x))\n\\]\n\nThis property makes it particularly useful in bac